In [2]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from fractions import Fraction
from decimal import *
TWOPLACES = Decimal(10) ** -2

In [3]:
# parameters
customer_id = '5'
formatted_attribute = 'front_facing_weight_capacity'
customer_name='%bedbathandbeyond%'
buckets = """Office Chairs"""

strategy_version_input=702
attribute_id_input=5147

In [4]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [5]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input}
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# buckets

In [6]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# _______________________________________________________________________________________________

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

# _______________________________________________________________________________________________

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
    
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
        
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))


def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))

measure='This is a fraction: 100 1/64'
[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

# #remove duplicates
# matches['matches']=[list(set(sublist)) for sublist in matches['matches']]

# >>> def flatten(t):
# ...     # Flatten list of lists
# ...     return [item for sublist in t for item in sublist]
# ... 
# >>> flatten([['silicon'],['steel','iron']])
# ['silicon', 'steel', 'iron']

['This is a fraction: 100.02']

In [47]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df = df[df['bucket_id'].isin(buckets)]
print(len(df))
regex_pattern=r'(?i)(?i)(.{40}front.?facing.{40})|()'
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[df['matches'].astype(str)!='[]']))
df=df[df['matches'].astype(str)!='[]'][0:500]
print(df['matches'].explode().value_counts())
df.head(10)

1684
128
 tether hook limits forward movement in front facing mode Tri-Stage cushions support infant'    6
ective comfort shoulder and bottom pads Front-facing seat Freestanding fold for easy transpo    6
d is ready to graduate from a rear to a front facing, simply turn the Peg Perego Convertible    6
anti-rebound bar. Converts from rear to front facing car seat Rear facing mode suitable for     6
hildren weighing 5 to 45 lb.; 47\" tall Front facing mode suitable for children weighing 22     6
space-saving design allows rear-facing, front-facing, and booster seat use. Handy features i    5
ing child. Converts from rear facing to front facing harness Rear facing mode suitable for c    4
ety, comfort, and style in rear-facing, front-facing, and booster modes for years of extende    4
 sound on-the-go Allows rear-facing and front-facing use Safely seats children weighing 5-40    4
hildren up to 33 lb. 3 carry positions: front facing inwards, front facing outwards and back    4
" W x 19\" 

product_id external_id  \
10    988561999    69536351   
13   1283261795    69675198   
16   1283252985    69726719   
26    988395568    66254416   
74    817419316    69524468   
77    817357457    67508358   
78    817353864    67313181   
104   817366660    67997435   
112   817559700    65080337   
123   817353619    67313204   

                                                           product_name  \
10            Safety 1ˢᵗ® Jive 2-in-1 Convertible Car Seat in Dark Grey   
13                         Safety 1st® Jive 2-in-1 Convertible Car Seat   
16                    Britax Emblem 3-Stage Convertible Car Seat, Slate   
26   Peg Perego® Primo Viaggio Convertible Kinetic Car Seat in Licorice   
74          Diono™ Radian 3 RXT All-In-One Convertible Car Seat in Blue   
77                            BRITAX® B-Lively Double Stroller in Raven   
78               BRITAX® Emblem 3-Stage Convertible Car Seat in Fushion   
104                        Baby Jogger® City Mini® 2 Stroller in Carbon   
112              Baby Trend® Navigator Lite Double Jogger in Candy Aple   
123             BRITAX® Allegiance 3-Stage Convertible Car Seat in Azul   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
10                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [44]:
regex_pattern_multi = r"(?i)(supports\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static load capacity)|(\d+\s?lb.?.?\s?capacity)|(store.?\s?up\s?to\s?\d+\s?lb)|(\d+\s?lb.?.?\s?static\s?weight\s?capacity)|(users\s?up\s?to\s\d+\s?lb)|(\d+\s?lb.?.?\s?max\s?capacity)|(\d+\s?lb.?.?\s?seating\s?capacity)|(hold.?\s?up\s?to\s?\d+\s?lb)|(Rated\s?up\s?to\s?\d+\s?lb)|(supports\s?\d+\s?lb)|(\d+\s?lb.?.?\s?break\s?strength)|(\d+\s?lb.?.?\s?Load\s?Capacity)|(weight capacity.?o?f?.?.?\s?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?weight capacity)|([^weighs]\s\d+.?\s?lb.?\s?weight capacity)|(\s\d*\,\d*\s?lb.?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?weight capacity)|(\s?\d*.?\d*\s?lb.?\s?d?i?s?t?r?i?b?u?t?e?d?\s?weight capacity)|(weight\s?capacity\s?of\s?\d+\s?lb)|(\s?\d+.?\d*\s?p?o?u?n?d?\s?weight capacity)|(p?o?u?n?d?\s?weight capacity\s?of\s?.?\s?\d+.?\d*\s?l?b?)|(m?a?x?i?m?u?m?\s?weight capacity.?\s?\d*.?\d*\s?lb)|([^weighs][^\)\.]\s\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\s?\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(\s\d*\,\d*\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|([^weighs]\d*\.\d\s?lb.?\s?m?a?x?i?m?u?m?\s?weight capacity)|(\s\d+\.\d+\s?lb.?\s?m?a?x?i?m?u?m?\s?weight\s?capacity)|(m?a?x?i?m?u?m?\s?weight\s?capacity.?\s?\(?s?e?a?t?i?n?g?\)?\s?.?\s?\d*\s?lb)|(\s\d+\s?.?\s?m?a?x?i?m?u?m?\s?weight capacity)|()"               
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_multi, x))
print(len(df[df['matches'].astype(str)!='[]']))
df['matches'].explode().value_counts()

8


maximum weight capacity: 110 lb    3
holds up to 10 lb                  3
maximum weight capacity: 90 lb     1
 weight capacity: 90 lb            1
Name: matches, dtype: int64

In [35]:
deep['matches'].explode().value_counts()

 tether hook limits forward movement in front facing mode Tri-Stage cushions support infant'    6
ective comfort shoulder and bottom pads Front-facing seat Freestanding fold for easy transpo    6
anti-rebound bar. Converts from rear to front facing car seat Rear facing mode suitable for     6
hildren weighing 5 to 45 lb.; 47\" tall Front facing mode suitable for children weighing 22     6
d is ready to graduate from a rear to a front facing, simply turn the Peg Perego Convertible    6
space-saving design allows rear-facing, front-facing, and booster seat use. Handy features i    5
 sound on-the-go Allows rear-facing and front-facing use Safely seats children weighing 5-40    4
ety, comfort, and style in rear-facing, front-facing, and booster modes for years of extende    4
ldren weighing 5 to 40 lb. Front facing mode suitable for c                                     4
ing child. Converts from rear facing to front facing harness Rear facing mode suitable for c    4
hildren up to 33 lb.

In [14]:
deep['matches']= deep['matches'].apply(lambda x: re.sub(r'(?i)\\" deep|"deep|”\s?deep','',str(x))).apply(lambda x: re.sub(r'\\" deep|"deep|\"xa0deep|xa0deep|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

In [15]:
regex_pattern_comma=r','
deep['comma_matches'] = deep['matches'].apply(lambda x: re_extract(regex_pattern_comma, str(x)))
final_deep=deep[deep['comma_matches'].astype(str)=='[]']
final_deep.head()

product_id external_id  \
0   817679289    69325960   
1  1185204908    69688099   
2  1346491755    69726685   
3   988490201    69632540   
4  1294064743    69689043   

                                               product_name  \
0        BABYBJÖRN® 3D Mesh Baby Carrier Free in Anthracite   
1  Moby® Wrap Cloud Ultra-Light Hybrid Carrier in High Rise   
2              Stokke® Xplory® X Carry Cot in Golden Yellow   
3             UPPAbaby® Cruz V2 PiggyBack® Ride-Along Board   
4                UPPAbaby® VISTA® V2 RumbleSeat® in Greyson   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                         The BABYBJÖRN 3D Mesh Baby Carrier Free allows you to carry your baby with ease and freedom. Providing comfortable wear for long periods, this breathable carrier features adjustable head support, leg positioning and length for a custom fit. Easy-to-use baby carrier with excellent comfort for you and baby Front and back carrying design Supports a minimum of 7 lb. up to 26.5 lb. 3 different carrying positions Head support, leg position and length are adjustable so baby gets the right support and sits comfortably Integrated hood Breathable 3D mesh fabric keeps your baby cool and comfortable 100% polyester Padded, adjustable head support Built-in back support, waist belt and padded shoulder straps provide pressure relief Machine wash Limited lifetime manufacturer's warranty Imported Model 099013CA Anthracite; 099018CA Silver; 099038CA Sage Green   
1  This sleek and easy-to-use Moby Wrap Cloud Ultra-Light Hybrid Carrier keeps parents and baby stylish and comfortable when on-the-go. The light, airy design circulates airflow and its soft structure helps support baby's neck and torso. Supports children up to 33 lb. Created in a lightweight flex knit with the perfect amount of structure and airflow One-size fits most; friendly for plus-size and changing post-partum bodies Light, airy design circulates airflow with Powermesh fabric 2 ways to wear baby: inward-facing or world-facing Simple to slip on, adjust, and secure with sturdy buckles Softly structured design helps support baby’s neck and torso Mesh panels offer built-in support as baby grows Designed without fitted armholes Fabric tails tie to offer lower back support Integrated side pocket Pacifier clip Hands-free convenience Comfortable to wear for long periods of time Encourages bonding with baby Soft, flex-knit fabric with breathable Powermesh panels and 

In [17]:
# deep['matches'].apply(lambda x: re.sub(r'in| ','',str(x))).apply(lambda x:float(x))
# deep['matches'].apply(lambda x: f'"{x}"').apply(lambda x: re.sub(r',','","',str(x))).apply(lambda x: re.sub(r'," ',',"',str(x)))

In [27]:
final_deep[curation_col]=final_deep['matches']
matches=final_deep[['external_id','matches']]
matches['matches']=matches['matches'].apply(lambda x: float(x))
matches.head()

external_id  matches
12    63334760     15.0
27    63072075     15.0
28    63127058     16.0
35    67526277     15.0
36    67526215     15.0

In [19]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

#--------------------------------------------------------------------------------------------
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df=df[df['matches'].astype(str)!='[]']
# matches=df
# matches['matches'][0:20]

['13 to 17 Inches', '18 Inches and Higher', '3 to 12 Inches', 'n/a']

In [20]:
#--------------------------------------------------------------------------------------------
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df=df[df['matches'].astype(str)!='[]']
# matches=df
# matches['matches'][0:20]

None


In [21]:
metric=" Inches"
range_string =""""
13 - 17 
Greater than 18
3 - 12
"""
# range_string =""""

#  Greater than 108
#  12 - 23.9
#  24 - 35.9
#  36 - 47.9
#  48 - 59.9
#  60 - 71.9
#  72 - 83.9
#  84 - 95.9
#  96 - 107.9
#  0 - 11.9
#  """

In [22]:
string = range_string
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return f'{lower} to {upper}{metric}'
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        no_words=original_val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return f'{no_words}{metric} and Higher'
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >= 13 and num <= 17 :
        return f'13 to 17  Inches'
        
    if num >= 18:
        return f'18 Inches and Higher'
        
    if num >= 3 and num <= 12:
        return f'3 to 12 Inches'
        
    else:
        return r"n/a"



In [26]:
matches[curation_col]=matches['matches'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches.head()

external_id  matches    Q:depth_sheets
12    63334760     15.0  13 to 17  Inches
27    63072075     15.0  13 to 17  Inches
28    63127058     16.0  13 to 17  Inches
35    67526277     15.0  13 to 17  Inches
36    67526215     15.0  13 to 17  Inches

# na values

In [ ]:
df_na=df[df['matches'].astype(str)=='[]']
print(len(df_na))
df_na[curation_col]='n/a'
matches_na=df_na[['external_id',curation_col]]
df_na

# send to the folder for upload

In [32]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [33]:
looks_good('Bed Bath & Beyond', attribute, df, matches_na)